In [1]:
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:XPlot.Plotly.Interactive"

Installed Packages Microsoft.Data.Analysis, 0.20.1 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `/Users/mjajksj/.nuget/packages/xplot.plotly.interactive/4.0.6/interactive-extensions/dotnet/XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `/Users/mjajksj/.nuget/packages/microsoft.data.analysis/0.20.1/interactive-extensions/dotnet/Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using XPlot.Plotly;
using Microsoft.Data.Analysis;
using System.Linq;
using XPlot.Plotly.Interactive;

In [3]:
class Request
{
    public Request(int idx, int processingTime) {
        Id = idx;
        NeedProcessingTime = processingTime;
    }

    public int Id { get; set; }
    public int QueueTime { get; set; }
    public int SpentProcessingTime { get; set; }
    public int NeedProcessingTime { get; set; }

    public int AllTime => QueueTime + SpentProcessingTime;
}

In [4]:
class QS
{
    public Queue<Request> Queue { get; } 
    public List<Request> ProccessingRequests { get; }
    public List<Request> Processed { get; }
    
    public int ProcessorCount { get; }

    private int _freeProcessorsCounter = 0;

    public QS(int processorCount) {
        ProcessorCount = processorCount;
        Queue = new Queue<Request>();
        ProccessingRequests = new List<Request>();
        Processed = new List<Request>();
    }

    public void Tick() {
        while (ProccessingRequests.Count() < ProcessorCount) {
            if (Queue.TryDequeue(out var request)) {
                ProccessingRequests.Add(request);
            }
            else
            {
                break;
            }
        }

        var toProcessed = new List<Request>();
        foreach (var request in ProccessingRequests)
        {
            request.SpentProcessingTime += 1;
            if (request.SpentProcessingTime == request.NeedProcessingTime) {
                toProcessed.Add(request);
            }
        }

        _freeProcessorsCounter += (ProcessorCount - ProccessingRequests.Count());

        foreach (var request in toProcessed) {
            ProccessingRequests.Remove(request);
            Processed.Add(request);
        }

        foreach (var request in Queue)
        {
            request.QueueTime += 1;
        }
    }

    public Double AvgQueueTime(int t) => (Queue.Sum(x => x.QueueTime) + ProccessingRequests.Sum(x => x.QueueTime) + Processed.Sum(x => x.QueueTime)) / (double)(Queue.Count() + ProccessingRequests.Count() + Processed.Count());
    public Double AvgAllTime(int t) => (Queue.Sum(x => x.AllTime) + ProccessingRequests.Sum(x => x.AllTime) + Processed.Sum(x => x.AllTime)) / (double)(Queue.Count() + ProccessingRequests.Count() + Processed.Count());
    public Double AvgFreeTime(int t) => _freeProcessorsCounter / (double)ProcessorCount;
}

In [5]:
class RequestsGenerator 
{
    private int idx = 0;    
    private int maxTime;
    private int minTime;
    private int _l;
    private Random rnd = new Random();

    public RequestsGenerator(int l, int processingMinTime, int processingMaxTime) {
        maxTime = processingMaxTime;
        minTime = processingMinTime;
        _l = l;
    }

    public Request Next(int t)
    {
        if (t % _l == 0) {
            return new Request(++idx, rnd.Next(minTime, maxTime));
        }
        else {
            return null;
        }
    }
}

In [6]:
var requestsGenerator = new RequestsGenerator(40, 20, 40);
var qs = new QS(1);

In [7]:
var times = Enumerable.Range(0,500).ToList();
var queueTimes = new List<double>();
var freeTimes = new List<double>();
var allTimes = new List<double>();
var queueSize = new List<int>();
foreach (var t in times) {
    var request = requestsGenerator.Next(t);   
    if (request != null) {
        qs.Queue.Enqueue(request);
    }
    qs.Tick();
    queueTimes.Add(qs.AvgQueueTime(t));
    freeTimes.Add(qs.AvgFreeTime(t));
    allTimes.Add(qs.AvgAllTime(t));
    queueSize.Add(qs.Queue.Count());
}

In [8]:
var scatterQueue = new Scatter
{   
    x = times,
    y = queueTimes,
    marker = new Marker { color = "red" },
    name = "Среднее время в очереди"
};

var scatterFree = new Scatter
{   
    x = times,
    y = freeTimes,
    marker = new Marker { color = "blue" },
    name = "Среднее время в простоя"
};

var scatterAll = new Scatter
{   
    x = times,
    y = allTimes,
    marker = new Marker { color = "green" },
    name = "Среднее время в системе"
};

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "Время существования системы, (тики)",
    },
    yaxis = new Yaxis
    {
        title = "Среднее время, (тики)"
    },
    title = $"Средние времена"
};

Chart.Plot(new List<Trace>() { scatterQueue, scatterFree, scatterAll }, layout)

<!DOCTYPE html>

In [9]:
var scatter = new Scatter
{   
    x = times,
    y = queueSize,
    marker = new Marker { color = "red" },
    name = "Размер очереди"
};

var layout = new Layout.Layout 
{
    xaxis = new Xaxis 
    {
        title = "Время существования системы, (тики)",
    },
    yaxis = new Yaxis
    {
        title = "Размер очереди, (чел.)"
    },
    title = $"Размер очереди"
};

Chart.Plot(new List<Trace>() { scatter }, layout)

<!DOCTYPE html>

In [10]:
display($"Средний размер очереди: {queueSize.Sum() / (double)times.Count()} человека")

Средний размер очереди: 0 человека

In [11]:
display($"Максимальный размер очереди: {queueSize.Max()} человека")

Максимальный размер очереди: 0 человека